# Multi-Season & Multi-Target Prediction

This notebook generates comprehensive predictions for **2025 AND 2026**.

### 🧠 Models Used:
1.  **XGBoost Regressor:** For exact finishing position (P1-P20).
2.  **LightGBM Classifier (Win):** Probability of P1.
3.  **LightGBM Classifier (Podium):** Probability of Top 3.
4.  **LightGBM Classifier (Top 5):** Probability of Top 5.
5.  **LightGBM Classifier (Points):** Probability of Top 10.

### ⚙️ Logic:
- **Past Races:** We use ACTUAL Qualifying results to generate the prediction (allowing Side-by-Side comparison).
- **Future Races:** We SIMULATE Qualifying based on driver form.


In [1]:
import pandas as pd
import numpy as np
import joblib
import json
import os
import sys
from datetime import datetime, timedelta
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

# Add path to backend
sys.path.append(os.path.abspath('../../'))

from backend.app.core.config import settings
sync_db_url = settings.DATABASE_URL.replace("postgresql+asyncpg", "postgresql")
engine = create_engine(sync_db_url)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)


In [2]:
# --- LOAD MODELS ---
MODEL_DIR = "training/v3/models"
print("📥 Loading Model Suite...")

try:
    xgb_pos = joblib.load(os.path.join(MODEL_DIR, "XGB_Regressor_Position_Baseline_v3_20251102_232011.pkl"))
    lgbm_win = joblib.load(os.path.join(MODEL_DIR, "LGBM_Classifier_win_Baseline_v3_20251102_220712.pkl"))
    lgbm_podium = joblib.load(os.path.join(MODEL_DIR, "LGBM_Classifier_podium_Optimized_v3_20251102_220712.pkl"))
    lgbm_top5 = joblib.load(os.path.join(MODEL_DIR, "LGBM_Classifier_top5_Baseline_v3_20251102_220712.pkl"))
    lgbm_points = joblib.load(os.path.join(MODEL_DIR, "LGBM_Classifier_points_finish_Baseline_v3_20251102_220712.pkl"))
    
    circuit_encoder = joblib.load(os.path.join(MODEL_DIR, "encoders_new", "circuit_encoder.pkl"))
    
    with open(os.path.join(MODEL_DIR, "XGB_Regressor_Position_Baseline_v3_20251102_232011_metadata.json")) as f:
        meta = json.load(f)
        feature_cols = meta['features']

    print("✅ All Models & Metadata Loaded Successfully")
except Exception as e:
    print(f"❌ Error loading models: {e}")


📥 Loading Model Suite...
✅ All Models & Metadata Loaded Successfully


In [3]:
# --- FIX: ENSURE 2026 SEASON IS CORRECT ---
def ensure_2026_season_exists():
    with engine.connect() as conn:
        # 1. Delete existing 2026 data to fix the 'is_completed' bug
        conn.execute(text("DELETE FROM races WHERE season = 2026"))
        conn.commit()
        print("🔄 Resetting 2026 Season...")

        print("📅 Creating 2026 Season (Cloning 2025)...")
        races_25 = pd.read_sql(text("SELECT * FROM races WHERE season = 2025"), conn)
        
        session = SessionLocal()
        try:
            for _, row in races_25.iterrows():
                new_date = row['race_date'] + timedelta(weeks=52) if row['race_date'] else None
                
                insert_query = text("""
                    INSERT INTO races (season, round_number, race_name, circuit_name, circuit_location, country, race_date, is_completed)
                    VALUES (2026, :round, :name, :circ, :loc, :country, :date, 0)
                """)
                session.execute(insert_query, {
                    "round": row['round_number'],
                    "name": row['race_name'],
                    "circ": row['circuit_name'],
                    "loc": row['circuit_location'],
                    "country": row['country'],
                    "date": new_date
                })
            session.commit()
            print("✅ 2026 Season Created (All set to Upcoming)!")
        except Exception as e:
            print(f"❌ Error creating 2026 season: {e}")
            session.rollback()
        finally:
            session.close()

ensure_2026_season_exists()


🔄 Resetting 2026 Season...
📅 Creating 2026 Season (Cloning 2025)...
✅ 2026 Season Created (All set to Upcoming)!


In [4]:
def get_race_input_data(race_id, season, round_num, circuit_name, is_completed):
    
    with engine.connect() as conn:
        if is_completed:
            # 1. ACTUAL DATA (Comparison Mode)
            query = text("""
                SELECT 
                    r.driver_id, r.grid_position, d.code, t.id as team_id
                FROM race_results r
                JOIN drivers d ON r.driver_id = d.id
                JOIN teams t ON r.team_id = t.id
                WHERE r.race_id = :rid AND r.is_sprint = false
            """)
            drivers = pd.read_sql(query, conn, params={"rid": race_id})
            
            if len(drivers) > 0:
                drivers['grid_position'] = drivers['grid_position'].replace(0, 20).fillna(20)
                drivers['quali_position'] = drivers['grid_position']
                return drivers

        # 2. SIMULATED DATA (Forecast Mode)
        query = text("""
            SELECT d.id as driver_id, d.code, t.id as team_id
            FROM driver_seasons ds
            JOIN drivers d ON ds.driver_id = d.id
            JOIN teams t ON ds.team_id = t.id
            WHERE ds.season = :s
        """)
        drivers = pd.read_sql(query, conn, params={"s": season})
        
        # --- 🧠 IMPROVED DRIVER RATINGS ---
        driver_ratings = {
            'VER': 99,
            'NOR': 97,
            'LEC': 96,
            'PIA': 95,
            'HAM': 94,
            'RUS': 93,
            'SAI': 92,
            'ALO': 88,
            'TSU': 82,
            'ALB': 80,
        }
        
        drivers['skill'] = drivers['code'].map(driver_ratings).fillna(75)
        
        drivers['form'] = np.random.normal(0, 3, len(drivers))
        drivers['total_score'] = drivers['skill'] + drivers['form']
        
        drivers = drivers.sort_values('total_score', ascending=False).reset_index(drop=True)
        
        drivers['grid_position'] = range(1, len(drivers) + 1)
        drivers['quali_position'] = drivers['grid_position']
        
        return drivers


In [5]:
def predict_and_save(race_id, season, round_num, circuit_name, is_completed):
    
    df = get_race_input_data(race_id, season, round_num, circuit_name, is_completed)
    if len(df) == 0: return

    # --- FEATURE PREP ---
    features = pd.DataFrame(index=df.index)
    features['grid_position'] = df['grid_position']
    features['quali_position'] = df['quali_position']
    features['round_number'] = round_num
    features['driver_id'] = df['driver_id']
    features['team_id'] = df['team_id']
    
    features['weighted_points_5'] = 25 - (df['grid_position'] * 0.8)
    features['q3_gap_to_pole'] = (df['grid_position'] - 1) * 0.08
    features['avg_quali_position_5'] = df['grid_position']
    features['driver_championships'] = 0
    features['grid_penalty'] = 0
    features['is_street_circuit'] = 1 if 'Street' in circuit_name else 0
    features['is_long_circuit'] = 0
    features['has_sprint'] = 0
    
    try:
        c_enc = circuit_encoder.transform([circuit_name])[0]
    except:
        c_enc = 0
    features['circuit_encoded'] = c_enc
    
    for col in feature_cols:
        if col not in features.columns:
            features[col] = 0
            
    X = features[feature_cols]
    
    # --- PREDICT ---
    raw_scores = xgb_pos.predict(X)
    df['pred_pos'] = pd.Series(raw_scores).rank(method='first').astype(int)
    
    df['prob_win'] = lgbm_win.predict_proba(X)[:, 1]
    df['prob_podium'] = lgbm_podium.predict_proba(X)[:, 1]
    df['prob_top5'] = lgbm_top5.predict_proba(X)[:, 1]
    df['prob_points'] = lgbm_points.predict_proba(X)[:, 1]
    
    # --- SAVE ---
    session = SessionLocal()
    try:
        session.execute(text("DELETE FROM driver_predictions WHERE prediction_id IN (SELECT id FROM predictions WHERE race_id = :rid)"), {"rid": race_id})
        session.execute(text("DELETE FROM predictions WHERE race_id = :rid"), {"rid": race_id})
        
        res = session.execute(text("INSERT INTO predictions (race_id, model_version, created_at) VALUES (:rid, 'v5_smart_sim', NOW()) RETURNING id"), {"rid": race_id})
        pred_id = res.scalar()
        
        for _, row in df.iterrows():
            session.execute(text("""
                INSERT INTO driver_predictions 
                (prediction_id, driver_id, predicted_position, prob_win, prob_podium, prob_top5, prob_points, probability)
                VALUES (:pid, :did, :pos, :win, :pod, :top5, :pts, :win)
            """), {
                "pid": pred_id,
                "did": row['driver_id'],
                "pos": row['pred_pos'],
                "win": float(row['prob_win']),
                "pod": float(row['prob_podium']),
                "top5": float(row['prob_top5']),
                "pts": float(row['prob_points'])
            })
            
        session.commit()
        print(f"   ✅ {season} R{round_num}: Predictions saved.")
        
    except Exception as e:
        session.rollback()
        print(f"   ❌ Error saving R{round_num}: {e}")
    finally:
        session.close()


In [6]:
# --- EXECUTE ---
print("🚀 Generating V5 Predictions (Fixing Lando & 2026)...")

with engine.connect() as conn:
    races = pd.read_sql(text("SELECT * FROM races WHERE season >= 2024 ORDER BY season, round_number"), conn)

for _, race in races.iterrows():
    predict_and_save(race['id'], race['season'], race['round_number'], race['circuit_name'], race['is_completed'])


🚀 Generating V5 Predictions (Fixing Lando & 2026)...
   ✅ 2024 R1: Predictions saved.
   ✅ 2024 R2: Predictions saved.
   ✅ 2024 R3: Predictions saved.
   ✅ 2024 R4: Predictions saved.
   ✅ 2024 R5: Predictions saved.
   ✅ 2024 R6: Predictions saved.
   ✅ 2024 R7: Predictions saved.
   ✅ 2024 R8: Predictions saved.
   ✅ 2024 R9: Predictions saved.
   ✅ 2024 R10: Predictions saved.
   ✅ 2024 R11: Predictions saved.
   ✅ 2024 R12: Predictions saved.
   ✅ 2024 R13: Predictions saved.
   ✅ 2024 R14: Predictions saved.
   ✅ 2024 R15: Predictions saved.
   ✅ 2024 R16: Predictions saved.
   ✅ 2024 R17: Predictions saved.
   ✅ 2024 R18: Predictions saved.
   ✅ 2024 R19: Predictions saved.
   ✅ 2024 R20: Predictions saved.
   ✅ 2024 R21: Predictions saved.
   ✅ 2024 R22: Predictions saved.
   ✅ 2024 R23: Predictions saved.
   ✅ 2024 R24: Predictions saved.
   ✅ 2025 R1: Predictions saved.
   ✅ 2025 R2: Predictions saved.
   ✅ 2025 R3: Predictions saved.
   ✅ 2025 R4: Predictions saved.
   ✅ 202